# Airplane allocation problem - A stochastic problem

### Problem definition
An airline wishes to allocate airplanes of various types among its route to satisfy an uncertain passenger demand, in such a way as to minimize operating costs plus the lost revenue from passengers turned away.
This problem use *4* different types of aircraft and *5* routes

### Modelisation
In order to modelize the problem, we use a matrix $M$ containing the number of aircraft use for a given route and a given type :

$M = (x_{i,j})_{1\leq i\leq 5, 1\leq j\leq 4} = \left( \begin{array}{cccc}
x_{1,1} & \ldots & \ldots & x_{1,4} \\
x_{2,1} & \ldots & \ldots & x_{2,4} \\
x_{3,1} & \ldots & \ldots & x_{3,4} \\
x_{4,1} & \ldots & \ldots & x_{4,4} \\ 
x_{5,1} & \ldots & \ldots & x_{5,4} \\ \end{array} \right)$ where $x \in \mathbb{N}$

Where $x_{i,j}$ represents the number of airplane for the type $j$ on the route $i$.

Example $x_{3,4} = 23$ means that we have 23 aircrafts of type 4 on the route number 3

### Cost function

In order to find the best solution, we need to minimize the following cost function :

$f = \sum_{j=1}^{20} Cost_{j} * x_{j} + \sum_{k=1}^{5} RevenueLost_{k} * PassengersTA_{k}$

In [ ]:
### Resolution - Population algorithm
In order to find the best solution, we chosen the Genetic algorithm.


## The implementationn (Python)
### Initial imports

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import random
import copy

### The manipulated object
In a first place, we need to create an object with the following structure to store all the information

In [14]:
class PopulationMember:
    def __init__(self, airplane):
        # The airplane distribution of the individual
        self.airplane = airplane
        # Cost of the so called distribution
        self.cost = 0
        # 1/cost
        self.invert_cost = 0
        # Probability of being keep for the n+1 generation
        self.probability = 0

### Initials values

In [15]:
# All 5 routes numbered
routes = np.array([0, 1, 2, 3, 4])
# Revenue lost per routes if a passenger turned away
revenue_lost_per_passenger_turned_away = np.array([13, 20, 7, 7, 15])
# Demand in passenger for each route
passenger_demand_per_route = np.array([800, 900, 700, 650, 380])
# Number of aircraft available for each five types
number_of_aircraft_available_per_type = [10, 19, 25, 16]
# The aircraft capacity (row=type, column=route)
aircraft_capacity_per_spot = np.matrix([[16, 10, 30, 23],
                                        [16, 10, 30, 23],
                                        [16, 10, 30, 23],
                                        [16, 10, 30, 23],
                                        [16, 10, 30, 23]])
# Operational costs
operational_cost_per_spot = np.matrix([[12, 20, 30, 19],
                                       [2, 34, 10, 20],
                                       [43, 63, 40, 12],
                                       [32, 10, 6, 34],
                                       [20, 30, 10, 87]])
# Aircraft per spot (solution)
number_of_aircraft_per_spot = np.matrix([[0, 0, 0, 0],
                                         [0, 0, 0, 0],
                                         [0, 0, 0, 0],
                                         [0, 0, 0, 0],
                                         [0, 0, 0, 0]])

### Crossing function

In [16]:
def cross(parent1, parent2, cross_threshold):
    children = [PopulationMember(copy.deepcopy(parent1.airplane)), PopulationMember(copy.deepcopy(parent2.airplane))]
    row_nb, column_nb = number_of_aircraft_per_spot.shape
    for index in range(0, column_nb):
        np.random.seed(10)
        rand = random.uniform(0, 1)
        if rand <= cross_threshold:
            children[0].airplane[:, index] = copy.deepcopy(parent2.airplane[:, index])
            children[1].airplane[:, index] = copy.deepcopy(parent1.airplane[:, index])
    return children[0], children[1]

### Mutation funtion

In [17]:
def mutation(child, mutation_threshold):
    row_nb, column_nb = number_of_aircraft_per_spot.shape
    child_copy = copy.deepcopy(child)
    for index in range(0, column_nb):
        np.random.seed(10)
        rand = random.uniform(0, 1)
        if rand <= mutation_threshold:
            index1 = random.randint(0, row_nb - 1)
            index2 = random.randint(0, row_nb - 1)
            while index1 == index2:
                index2 = random.randint(0, row_nb - 1)
            tmp = child_copy.airplane[index1, index]
            child_copy.airplane[index1, index] = child_copy.airplane[index2, index]
            child_copy.airplane[index2, index] = tmp
    return child_copy